Crear un esquema externo referenciando a una base de datos previamente creada en glue:

In [0]:
drop schema if exists earth_temp_external_schema;
create external schema earth_temp_external_schema from data catalog 
database 'climate-change-db' 
iam_role 'arn:aws:iam::525046094193:role/LabRole' 
region 'us-east-1';

Como las tablas estan creadas desde Glue, puedo acceder a su informacion directamente:

In [0]:
select * from earth_temp_external_schema.by_country

Calculo de la temperatura minima y maxima de Albania

In [0]:
SELECT country, MIN(averagetemperature) as min_temp, MAX(averagetemperature) as max_temp FROM earth_temp_external_schema.by_country
WHERE country ='Albania'
GROUP BY country

Creacion de tabla nativa:

In [0]:
create schema if not exists earth_temp_local_schema;
drop table if exists earth_temp_local_schema.temp_by_country;
create table earth_temp_local_schema.temp_by_country(
dt date,
averagetemperature real,
averagetemperatureuncertainty real,
country varchar(200));

In [0]:
COPY earth_temp_local_schema.temp_by_country FROM 's3://climate-change-datalake/trusted/earth-temp/by_country/run-1677805729745-part-r-00000'
iam_role 'arn:aws:iam::525046094193:role/LabRole'
region 'us-east-1'
FORMAT AS CSV
IGNOREHEADER 1;

In [0]:
select * from earth_temp_local_schema.temp_by_country limit 10

Paises cuya temperatura promedio historica sea mayor a 20 grados.

In [0]:
select country, avg(averagetemperature) from earth_temp_local_schema.temp_by_country
Where averagetemperature >=20
group by country